In [2]:
!pip install youtube-comment-downloader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 7.9 MB/s eta 0:00:00


In [12]:
import os
import json
import pandas as pd

# 유튜브 URL 입력
url = input("유튜브 영상 링크를 입력하세요: ").strip()

# JSON 파일 이름
json_file = 'YoutubeComments.json'

# CSV 파일 이름
csv_file = 'YoutubeComments.csv'

# youtube-comment-downloader 명령어 실행
os.system(f'youtube-comment-downloader --url "{url}" --output {json_file}')

# JSON 파일 읽기
with open(json_file, 'r', encoding='utf-8') as f:
    json_data = [json.loads(line) for line in f]

# JSON 데이터를 DataFrame으로 변환
df = pd.DataFrame(json_data)

# CSV 파일로 저장
df.to_csv(csv_file, index=False, encoding='utf-8-sig')  # 한글 깨짐 방지

# JSON 파일 삭제
os.remove(json_file)

print(f"\n댓글이 {csv_file} 파일로 저장되었습니다.")


유튜브 영상 링크를 입력하세요: https://youtu.be/Oz-r4OvoE3s?list=PLZdTWwpnekj1NA7KeRzSinz08HYS6NzDr

댓글이 YoutubeComments.csv 파일로 저장되었습니다.


In [2]:
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=e13e5316c844102b9add501875ed0296a7f1412c8689187635dbe8f09991d00d
  Stored in directory: /root/.cache/pip/wheels/32/6a/b6/b7eb43a6ad55b139c15c5daa29f3707659cfa6944d3c696f5b
Successfully built langid


In [47]:
import pandas as pd
import langid
import re

# 파일 경로
file_path = 'YoutubeComments.csv'  # 코랩에 올린 파일명

# 댓글 불러오기
df = pd.read_csv(file_path)

# 1. 초성/은어 감지 함수
def is_korean_internet_slang(text):
    if pd.isna(text):
        return False
    text = str(text).strip()
    return (
        bool(re.fullmatch(r'[ㄱ-ㅎㅏ-ㅣㅜㅠㅋㅎ\s]+', text)) or
        'ㄹㅇ' in text or 'ㅇㅈ' in text or 'ㅅㅂ' in text or 'ㅋㅋㅋ' in text or 'ㄷㄷ' in text
    )

# 2. 전처리 함수 (특수문자 제거)
def clean_text(text):
    if pd.isna(text):
        return ''
    return re.sub(r'[^\w\sㄱ-힣]', '', str(text))

# 3. 최종 언어 감지 함수
def detect_langid_custom(text):
    try:
        text = str(text).strip()
        if len(text) < 2:
            return 'unknown'
        if is_korean_internet_slang(text):
            return 'ko'
        cleaned = clean_text(text)
        lang, _ = langid.classify(cleaned)
        return lang
    except:
        return 'unknown'

# 4. 언어 분류 실행
df['language'] = df['text'].apply(detect_langid_custom)

# 5. 언어별 분포 출력
print(df['language'].value_counts())

# 6. 결과 저장
df.to_csv('YoutubeComments_with_langid.csv', index=False, encoding='utf-8-sig')


language
en         385
ko         301
fr           9
es           9
ja           6
de           6
unknown      3
ms           3
pl           2
nl           2
zh           2
ur           2
la           2
ru           2
ro           1
kk           1
id           1
ca           1
pt           1
fi           1
sl           1
da           1
xh           1
et           1
lt           1
sv           1
Name: count, dtype: int64
